In [1]:
import numpy as np
import tensorflow as tf

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## TensorFlow Architecture

TensorFlow Core:
1. Building the computational graph (a tf.Graph)
2. Running the computational graph (using a tf.Session)

Graphs are composed of:
1. Operations (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
2. Tensors: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors. 

### Tensors

**Important: tf.Tensors do not have values, they are just handles to elements in the computation graph.**

A tensor's rank is its number of dimensions, while its shape is a tuple of integers specifying the array's length along each dimension.  TensorFlow uses numpy arrays to represent tensor values.

In [2]:
test1 = tf.constant([1., 2., 3.])
test2 = tf.constant([[1., 2., 3.], [4., 5., 6.]])
test3 = tf.constant([[[1., 2., 3.]], [[7., 8., 9.]]])

In [3]:
def get_rank_shape(x):
    return len(x.shape), x.shape.as_list()

print(get_rank_shape(test1))
print(get_rank_shape(test2))
print(get_rank_shape(test3))

(1, [3])
(2, [2, 3])
(3, [2, 1, 3])


### Sessions

**If a tf.Graph is like a .py file, a tf.Session is like the python executable.**

Notice that printing the tensors does not output the values 3.0, 4.0, and 7.0 as you might expect. The above statements only build the computation graph. These tf.Tensor objects just represent the results of the operations that will be run.

In [4]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

sess = tf.Session()
sess.run(total)

Tensor("Const_3:0", shape=(), dtype=float32)
Tensor("Const_4:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


7.0

During a call to tf.Session.run any tf.Tensor only has a single value. 

In [5]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.7269311  0.6736678  0.01711071]
[0.32206833 0.28396034 0.2040292 ]
(array([1.533049 , 1.7832601, 1.7190232], dtype=float32), array([2.533049 , 2.78326  , 2.7190232], dtype=float32))


### Feeding

A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later, like a function argument.

In [6]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


### Datasets

To get a runnable tf.Tensor from a Dataset you must first convert it to a tf.data.Iterator, and then call the Iterator's get_next method.  The simplest way to create an Iterator is with the make_one_shot_iterator method. 

In [7]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

[0 1]
[2 3]
[4 5]
[6 7]


Some TensorFlow functions return tf.Operations instead of tf.Tensors. The result of calling run on an Operation is None. You run an operation to cause a side-effect, not to retrieve a value. Examples of this include the initialization, and training ops demonstrated later.

In [8]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[-1.6156261  0.852058   1.3816661]
[ 0.05087419  0.02030041 -0.17740534]
[ 0.24847709  0.2564263  -2.766303  ]
[0.593479  0.7124496 1.3534671]
[-0.8896538 -0.5808319 -1.6039455]
[ 0.01796718 -0.04073609 -1.8054122 ]
[ 1.1821448 -1.8971509  0.8883202]
[-2.0514402  -0.8795718  -0.02998029]
[ 0.82727236 -0.13169971 -0.76910526]
[-0.13983044 -0.9151707  -0.4931749 ]


### Layers

A trainable model must modify the values in the graph to get new outputs with the same input. Layers are the preferred way to add trainable parameters to a graph.

Layers package together both the variables and the operations that act on them. For example a dense (or fully-connected) layer performs a weighted sum across all inputs for each output and applies an optional activation function. The connection weights and biases are managed by the layer object.

The following code creates a Dense layer that takes a batch of input vectors, and produces a single output value for each. To apply a layer to an input, call the layer as if it were a function.  The layer inspects its input to determine sizes for its internal variables. So here we must set the shape of the x placeholder so that the layer can build a weight matrix of the correct size.

In [9]:
x = tf.placeholder(tf.float32)#, shape=[None, 3])
linear_model = tf.layers.Dense(units=1, kernel_initializer=tf.zeros_initializer, bias_initializer=tf.constant_initializer(1))
y = linear_model(x)

ValueError: Input 0 of layer dense_1 is incompatible with the layer: its rank is undefined, but the layer requires a defined rank.

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1, kernel_initializer=tf.constant_initializer([1, 2, 3]), 
                               bias_initializer=tf.constant_initializer(1))
y = linear_model(x)

The layer contains variables that must be initialized before they can be used. 

A TensorFlow variable is the best way to represent shared, persistent state manipulated by your program.  Variables are manipulated via the tf.Variable class. A tf.Variable represents a tensor whose value can be changed by running ops on it. Unlike tf.Tensor objects, a tf.Variable exists outside the context of a single session.run call.

**Important: Calling tf.global_variables_initializer only creates and returns a handle to a TensorFlow operation. That op will initialize all the global variables when we run it with tf.Session.run.**

**Also note that this global_variables_initializer only initializes variables that existed in the graph when the initializer was created. So the initializer should be one of the last things added during graph construction.**

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
sess.run(linear_model.weights)

In [ ]:
sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]})

### Feature Columns

The easiest way to experiment with feature columns is using the tf.feature_column.input_layer function. This function only accepts dense columns as inputs, so to view the result of a categorical column you must wrap it in an tf.feature_column.indicator_column.

Definition of dense column??

In [ ]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']
}

department_column = tf.feature_column.categorical_column_with_vocabulary_list('department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

Feature columns can have internal state, like layers, so they often need to be initialized. Categorical columns use lookup tables internally and these require a separate initialization op, tf.tables_initializer.

In [ ]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))
sess.run(inputs)